In [2]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

# Load Data

In [3]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Is transformed to be better with VGG16 model
    ])

In [4]:
def getDataset(train=True, sample_size=100):
    dataset = CIFAR10(root='./data',
                  train=train, 
                  download=True,
                  transform=transform)
    
    X = []
    y = []

    for i in range(sample_size):
        X.append( dataset[i][0] )
        y.append( dataset[i][1] )

    X = torch.stack( X )

    return X, y

X_train, y_train = getDataset() #Training data
X_test, y_test = getDataset(train=False) #Test data

Files already downloaded and verified
Files already downloaded and verified


# Create Minibatch

In [5]:
batchSize = 32

In [6]:
indices = np.random.choice(X_train.shape[0], batchSize, replace=False) # Not used as of now
minibatch = X_train[indices]
#minibatch

# Load Model

In [7]:
from torchvision import models
pretrained_model = models.vgg16(pretrained=True)
pretrained_model.classifier = torch.nn.Identity() # 25088 features output

c:\Users\tobop\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tobop\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(25088, 10000),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(10000, 4000)    # Second fully connected layer to reduce to 4000
        )

    def forward(self, x):
        return self.fc_layers(x)

In [9]:
class CombinedModel(nn.Module):
    def __init__(self, cnn_extractor, custom_nn):
        super(CombinedModel, self).__init__()
        self.cnn_extractor = cnn_extractor  # Pretrained CNN (frozen)
        self.custom_nn = custom_nn  # Custom NN (trainable)

    def forward(self, x):
        features = self.cnn_extractor(x)  # Extract features using the CNN
        output = self.custom_nn(features)  # Pass the features through the custom NN
        return output


In [10]:
custom_nn = CustomNN()
model = CombinedModel(pretrained_model, custom_nn)

# Forward Propagate Through Model

In [11]:
result = model(X_train) # torch.Size([100, 4000])

In [13]:
# Parameters
mean = 0
std_dev = np.sqrt(0.01)  # Standard deviation is the square root of the variance

# Create the matrix W and V with shape (4000, 32) and (32, 1) - SHOULD BE A FUNCTION WHERE 32 IS THE NUMBER OF BITS WE WANT
W = torch.normal(mean, std_dev, (4000, 32))
V = torch.normal(mean, std_dev, (32, 1)) 

print(W.shape)
print(V.shape)

torch.Size([4000, 32])
torch.Size([32, 1])


In [14]:
U = []
for i in range(len(result)):
    dot = torch.matmul(W.T, result[i])
    dot = dot.reshape(32, 1)
    u = (dot + V)
    U.append(u)
U = torch.stack(U) # torch.Size([100, 32, 1])
U = U.reshape(100, 32)

In [16]:
# Compute the dot product matrix -> Useful for finding Theta in loss function
dot_product_matrix = torch.matmul(U, U.T)
dot_product_matrix #(100, 100) Shape -> (sample_size, sample_size) Shape

#Calculate Theta
Theta = 1/2 * dot_product_matrix

# Finding S Based On Labels

In [17]:
y_train = torch.tensor(y_train) # (sample_size)
S = (y_train[:, None] == y_train).float() # (sample_size, sample_size) Shape -> 1 if labels are the same, 0 if not

torch.Size([100])

# Finding binary codes B

In [20]:
#Find the Binary codes
B = torch.sign(U)

## Custom Loss Function

In [11]:
#Parameters
eta = 0.25 #Learning Rate

In [12]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self, mean, std_dev, eta = 0.25, hash_length = 32):
        super(CustomLoss, self).__init__()
        # Initialize W and V as learnable parameters
        self.W = nn.Parameter(torch.normal(mean, std_dev, (4000, hash_length)))  # W will be updated during training
        self.V = nn.Parameter(torch.normal(mean, std_dev, (hash_length, 1)))     # V will be updated during training
        self.eta = eta  # Regularization parameter

    def forward(self, inputs, targets, hash_length = 32):
        targets = torch.tensor(targets)
        S = (targets[:, None] == targets).float() # S calculation
        U = (torch.matmul(self.W.T, inputs) + self.V).reshape(100, hash_length) # U calculation

        #Calculate Theta
        dot_product_matrix = torch.matmul(U, U.T)
        dot_product_matrix # (sample_size, sample_size) Shape
        Theta = 1/2 * dot_product_matrix

        #Calculate hash codes
        B = torch.sign(U)

        loss = - torch.sum(S * Theta - torch.log(1 + torch.exp(Theta)))
        loss += + self.eta * torch.sum(torch.norm(B - U, dim = 1).pow(2))    
        return loss.mean()


# Using Custom Loss Function

In [13]:
num_epochs = 10

In [14]:
loss_fn = CustomLoss(mean=0, std_dev=0.01, eta=0.25)


for param in pretrained_model.parameters(): #Lås parameters i VGG16
    param.requires_grad = False

optimizer = optim.Adam(model.custom_nn.parameters(), lr=0.001)


for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients
    inputs = X_train  # Your input data
    targets = y_train  # Your target labels
    outputs = model(inputs)  # Forward pass through the combined model
    loss = loss_fn(outputs, targets)  # Compute the custom loss
    loss.backward()  # Backpropagate
    optimizer.step()  # Update only the custom NN's parameters

NameError: name 'W' is not defined